# Iron Condor Strategy Backtest

This notebook backtests the Iron Condor strategy on SPY.

## Strategy Overview
- Sell OTM call and put credit spreads
- Target IV Rank > 40
- Range-bound markets
- 30-45 DTE
- Take profit at 50%, stop loss at 200%

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

from strategies import IronCondorStrategy
from data_providers import YFinanceProvider

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Data

In [ ]:
# Fetch SPY data
ticker = "SPY"
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 12, 31)

provider = YFinanceProvider()
data = provider.get_ohlcv([ticker], start_date=start_date, end_date=end_date)
df = data[ticker]

print(f"Loaded {len(df)} days of data for {ticker}")
df.head()

## Calculate Historical Volatility & IV Rank Proxy

In [ ]:
# Calculate returns
df['returns'] = df['close'].pct_change()

# Calculate 20-day rolling volatility (annualized)
df['hv_20'] = df['returns'].rolling(window=20).std() * np.sqrt(252)

# Calculate IV Rank proxy (using HV range over 252 days)
def calculate_ivr(hv_series, window=252):
    ivr = []
    for i in range(len(hv_series)):
        if i < window:
            ivr.append(50.0)  # Default
        else:
            lookback = hv_series[i-window:i]
            current = hv_series.iloc[i]
            min_hv = lookback.min()
            max_hv = lookback.max()
            if max_hv > min_hv:
                rank = ((current - min_hv) / (max_hv - min_hv)) * 100
                ivr.append(rank)
            else:
                ivr.append(50.0)
    return ivr

df['ivr'] = calculate_ivr(df['hv_20'])

# Plot IVR
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['ivr'], label='IV Rank', color='purple')
plt.axhline(y=40, color='green', linestyle='--', label='Entry Threshold (40)')
plt.title('IV Rank Over Time (Proxy using HV)')
plt.xlabel('Date')
plt.ylabel('IV Rank')
plt.legend()
plt.grid(True)
plt.show()

## Simplified Backtest

Since we don't have historical options data, we'll simulate:
- Enter when IVR > 40
- Assume credit received = 2% of underlying price
- Assume max loss = 5% of underlying (spread width)
- Hold for 30 days or until profit/loss target hit

In [ ]:
# Backtest parameters
ivr_threshold = 40
credit_pct = 0.02  # 2% credit
max_loss_pct = 0.05  # 5% max loss
take_profit_pct = 0.50  # Close at 50% profit
stop_loss_pct = 2.00  # Stop at 200% loss (2x credit)
hold_days = 30

# Track trades
trades = []
in_trade = False
entry_date = None
entry_price = None
credit_received = None

for i in range(len(df)):
    date = df.index[i]
    price = df['close'].iloc[i]
    ivr = df['ivr'].iloc[i]
    
    # Check if in trade
    if in_trade:
        days_in_trade = (date - entry_date).days
        
        # Calculate P&L (simplified)
        # If price stays in range, we keep credit
        # If price moves too much, we lose
        price_change_pct = abs(price - entry_price) / entry_price
        
        if price_change_pct > max_loss_pct:
            # Hit max loss
            pnl = -credit_received * stop_loss_pct
            trades.append({
                'entry_date': entry_date,
                'exit_date': date,
                'entry_price': entry_price,
                'exit_price': price,
                'credit': credit_received,
                'pnl': pnl,
                'outcome': 'stop_loss',
                'days': days_in_trade,
            })
            in_trade = False
        elif days_in_trade >= hold_days:
            # Hold period expired - take profit
            pnl = credit_received * take_profit_pct
            trades.append({
                'entry_date': entry_date,
                'exit_date': date,
                'entry_price': entry_price,
                'exit_price': price,
                'credit': credit_received,
                'pnl': pnl,
                'outcome': 'take_profit',
                'days': days_in_trade,
            })
            in_trade = False
    
    # Check for entry
    elif ivr > ivr_threshold and not in_trade:
        # Enter trade
        in_trade = True
        entry_date = date
        entry_price = price
        credit_received = price * credit_pct

print(f"\nBacktest complete: {len(trades)} trades")

## Analyze Results

In [ ]:
if trades:
    trades_df = pd.DataFrame(trades)
    
    print("\n=== Iron Condor Backtest Results ===")
    print(f"Total Trades: {len(trades_df)}")
    print(f"Winning Trades: {len(trades_df[trades_df['pnl'] > 0])}")
    print(f"Losing Trades: {len(trades_df[trades_df['pnl'] < 0])}")
    print(f"Win Rate: {len(trades_df[trades_df['pnl'] > 0]) / len(trades_df) * 100:.1f}%")
    print(f"\nTotal P&L: ${trades_df['pnl'].sum():.2f}")
    print(f"Average P&L per Trade: ${trades_df['pnl'].mean():.2f}")
    print(f"Average Winner: ${trades_df[trades_df['pnl'] > 0]['pnl'].mean():.2f}")
    print(f"Average Loser: ${trades_df[trades_df['pnl'] < 0]['pnl'].mean():.2f}")
    print(f"\nAverage Hold Time: {trades_df['days'].mean():.1f} days")
    
    # Equity curve
    trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()
    
    plt.figure(figsize=(14, 6))
    plt.plot(trades_df['exit_date'], trades_df['cumulative_pnl'], marker='o')
    plt.title('Iron Condor Strategy - Cumulative P&L')
    plt.xlabel('Date')
    plt.ylabel('Cumulative P&L ($)')
    plt.grid(True)
    plt.show()
    
    # Trade distribution
    plt.figure(figsize=(10, 6))
    plt.hist(trades_df['pnl'], bins=20, edgecolor='black', alpha=0.7)
    plt.axvline(x=0, color='red', linestyle='--', label='Break Even')
    plt.title('P&L Distribution')
    plt.xlabel('P&L per Trade ($)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No trades generated in backtest period")

## Conclusion

This simplified backtest shows the Iron Condor strategy's performance on SPY.

**Note**: This is a simplified simulation. Real backtesting would require:
- Historical options data (strikes, prices, greeks)
- Precise entry/exit pricing
- Commission and slippage modeling
- Event filtering (earnings, FOMC)
- Portfolio-level risk management

For production backtests, use services like OptionMetrics or CBOE historical data.